#### CIS435: Practical Data Science with Machine Learning | Assignment 3
## Clustering for Customer Segmentation in Marketing

Author: Mark Cody | Date: February 28, 2021

Purpose: Using customer data and clustering machine learning algorithms, determine customer segments to inform the development of marketing strategy. 

The Credit Card Dataset for Clustering is retrieved from: https://www.kaggle.com/arjunbhasin2013/ccdata/activity

In [1]:
%matplotlib inline
from copy import deepcopy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

### Data Understanding and Quality

In [2]:
# Importing the dataset
data = pd.read_csv('cadata.csv')
print("Number of rows and columns: {}".format(data.shape))
data.info()

Number of rows and columns: (6906, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6906 entries, 0 to 6905
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           6906 non-null   object 
 1   BALANCE                           6906 non-null   float64
 2   BALANCE_FREQUENCY                 6906 non-null   float64
 3   PURCHASES                         6906 non-null   float64
 4   ONEOFF_PURCHASES                  6906 non-null   float64
 5   INSTALLMENTS_PURCHASES            6906 non-null   float64
 6   PURCHASES_FREQUENCY               6906 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        6906 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  6906 non-null   float64
 9   PURCHASES_TRX                     6906 non-null   int64  
 10  CREDIT_LIMIT                      6906 non-null   float64
 11  PAYMENTS                      

In [6]:
data.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.166667,0.000000,0.083333,2,1000.0,201.802084,139.509787,0.0,12
1,C10003,2495.148862,1.000000,773.17,773.17,0.00,1.000000,1.000000,0.000000,12,7500.0,622.066742,627.284787,0.0,12
2,C10004,1666.670542,0.636364,1499.00,1499.00,0.00,0.083333,0.083333,0.000000,1,7500.0,0.000000,NaN,0.0,12
3,C10005,817.714335,1.000000,16.00,16.00,0.00,0.083333,0.083333,0.000000,1,1200.0,678.334763,244.791237,0.0,12
4,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.666667,0.000000,0.583333,8,1800.0,1400.057770,2407.246035,0.0,12


In [3]:
data.isnull().any()

CUST_ID                             False
BALANCE                             False
BALANCE_FREQUENCY                   False
PURCHASES                           False
ONEOFF_PURCHASES                    False
INSTALLMENTS_PURCHASES              False
PURCHASES_FREQUENCY                 False
ONEOFF_PURCHASES_FREQUENCY          False
PURCHASES_INSTALLMENTS_FREQUENCY    False
PURCHASES_TRX                       False
CREDIT_LIMIT                        False
PAYMENTS                            False
MINIMUM_PAYMENTS                     True
PRC_FULL_PAYMENT                    False
TENURE                              False
dtype: bool

In [9]:
#A quality problem exists where the customer has not made a minimum payment, 
#but the values are recorded as NaN. NaN should be replaced with 0. 
#An example follows: 
data[data['CUST_ID']== 'C10323']

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
258,C10323,515.904786,1.0,5359.02,2399.78,2959.24,1.0,1.0,1.0,60,6000.0,0.0,NaN,0.0,12


In [10]:
data['MINIMUM_PAYMENTS'] = data['MINIMUM_PAYMENTS'].fillna(0)

In [11]:
data['MINIMUM_PAYMENTS'].isnull().any()

False

In [12]:
#There are no duplicate values.
data.duplicated().value_counts()

False    6906
dtype: int64

#### Exploratory Data Analysis (EDA)

In [14]:
#EDA as a stand-alone report with sweetviz.
#Upon running this cell the report appears in a new browser tab.
import sweetviz as sv
data_report = sv.analyze(data)
data_report.show_html('EDA_Report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report EDA_Report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [15]:
data.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,6906.000000,6906.000000,6906.00000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000,6906.000000
mean,1391.124294,0.875828,1300.01406,767.783700,532.606509,0.635265,0.262368,0.472084,19.063568,4630.069717,1752.710455,794.019325,0.186019,11.576021
std,2037.957692,0.238088,2351.74969,1853.692453,997.578372,0.341552,0.315645,0.392193,26.791844,3729.790198,2927.401991,2259.037166,0.317224,1.263456
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,150.000000,0.000000,0.000000,0.000000,6.000000
25%,85.064043,0.875000,243.01000,0.000000,0.000000,0.333333,0.000000,0.000000,5.000000,1700.000000,391.271838,152.424900,0.000000,12.000000
50%,609.715242,1.000000,594.79000,194.335000,233.310000,0.666667,0.100000,0.416667,11.000000,3500.000000,885.157419,229.829030,0.000000,12.000000
75%,1773.145345,1.000000,1471.22250,854.537500,627.340000,1.000000,0.416667,0.888889,23.000000,6500.000000,1942.046036,735.300043,0.250000,12.000000
max,19043.138560,1.000000,49039.57000,40761.250000,22500.000000,1.000000,1.000000,1.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


#### Data Engineering
We begin by engineering the data, converting a Pandas DataFrame to a Numpy Array. We must also exclude string attributes (CUST_ID) as the algorithm requires the float datatype.

In [30]:
# Define features, excluding customer ID
f01 = data['BALANCE'].values
f02 = data['BALANCE_FREQUENCY'].values
f03 = data['PURCHASES'].values
f04 = data['ONEOFF_PURCHASES'].values
f05 = data['INSTALLMENTS_PURCHASES'].values
f06 = data['PURCHASES_FREQUENCY'].values
f07 = data['ONEOFF_PURCHASES_FREQUENCY'].values
f08 = data['PURCHASES_INSTALLMENTS_FREQUENCY'].values
f09 = data['PURCHASES_TRX'].values
f10 = data['CREDIT_LIMIT'].values
f11 = data['PAYMENTS'].values
f12 = data['MINIMUM_PAYMENTS'].values
f13 = data['PRC_FULL_PAYMENT'].values
f14 = data['TENURE'].values

In [33]:
X = np.array(list(zip(f01,f02,f03,f04,f05,f06,f07,f08,f09,f10,f11,f12,f13,f14)))

## Machine Learning Models 

In [28]:
# We will use the Sci-Kit Learning library and import KMeans.
from sklearn.cluster import KMeans

In [34]:
# Name the model, instantiate the algorithm, and specify the number of clusters (4).
kmeans = KMeans(n_clusters=4)
# Fit the model to the data
kmeans = kmeans.fit(X)

In [44]:
# The model will identify the cluster with integer labels (0,1,2,3).
labels = kmeans.predict(X)
print(labels[1000:1100]) #Below is an excerpt of the labels from 1000 to 1100.

[0 1 3 1 0 1 0 2 2 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 2 1 1 0 2 0 0 1 0 0 0 1
 0 1 1 0 1 1 1 0 2 0 2 1 2 0 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 2 1 0 1 1 0
 1 0 2 0 0 1 0 2 0 2 1 0 1 1 0 0 0 0 0 1 1 1 2 1 0 0]


In [38]:
# The model with calculate the center of each cluster, called a "centroid."
centroids = kmeans.cluster_centers_
print(centroids) #The centroid contains a value for each of the 14 attributes. 

[[1.95449873e+03 9.11217706e-01 1.38902331e+03 8.36660089e+02
  5.52826257e+02 6.60561049e-01 3.60706137e-01 4.53170470e-01
  2.10143778e+01 7.74382071e+03 1.86066060e+03 7.20763511e+02
  1.92607880e-01 1.17615270e+01]
 [6.96601552e+02 8.48544311e-01 7.13177815e+02 3.55726694e+02
  3.57832536e+02 6.01342605e-01 1.80725644e-01 4.60517657e-01
  1.30953691e+01 2.36190292e+03 9.54051829e+02 5.76996682e+02
  1.81078818e-01 1.14530032e+01]
 [4.96337281e+03 9.66606433e-01 4.79268932e+03 3.05773463e+03
  1.73495469e+03 8.14935710e-01 5.48363274e-01 6.36364521e-01
  5.78011928e+01 1.12671065e+04 6.99697150e+03 2.85180482e+03
  1.86924744e-01 1.18827038e+01]
 [5.40533094e+03 9.57954542e-01 2.72763637e+04 2.18771029e+04
  5.39926083e+03 9.09027750e-01 8.42361167e-01 7.20833292e-01
  1.27958333e+02 1.60833333e+04 2.79256345e+04 3.26667104e+03
  5.11205792e-01 1.19166667e+01]]


### Customer Cluster Assignment

In [47]:
#Add the cluster lables to the dataset. 
data['Clusters'] = labels

In [48]:
data.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,Clusters
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.166667,0.000000,0.083333,2,1000.0,201.802084,139.509787,0.0,12,1
1,C10003,2495.148862,1.000000,773.17,773.17,0.00,1.000000,1.000000,0.000000,12,7500.0,622.066742,627.284787,0.0,12,0
2,C10004,1666.670542,0.636364,1499.00,1499.00,0.00,0.083333,0.083333,0.000000,1,7500.0,0.000000,0.000000,0.0,12,0
3,C10005,817.714335,1.000000,16.00,16.00,0.00,0.083333,0.083333,0.000000,1,1200.0,678.334763,244.791237,0.0,12,1
4,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.666667,0.000000,0.583333,8,1800.0,1400.057770,2407.246035,0.0,12,1


### Cluster Characteristics

In [110]:
#How many customers were assigned to each cluster?
cluster_count = data['Clusters'].value_counts()
print(cluster_count)

1    4362
0    2016
2     504
3      24
Name: Clusters, dtype: int64


In [118]:
#What percent of the total does each group represent?
prc_ = round(cluster_count/len(data),3)
prc_

1    0.632
0    0.292
2    0.073
3    0.003
Name: Clusters, dtype: float64

##### Descriptive Statistics of Each Group

In [57]:
c3 = data[data['Clusters']==3].describe()
c3

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,Clusters
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.0
mean,5405.330935,0.957955,27276.363750,21877.102917,5399.260833,0.909028,0.842361,0.720833,127.958333,16083.333333,27925.634496,3266.671038,0.511206,11.916667,3.0
std,4317.955150,0.151737,8585.708142,8649.208460,4754.823099,0.223768,0.241322,0.373657,87.801891,5571.017132,9111.539978,5011.401903,0.405746,0.408248,0.0
min,1268.809031,0.272727,12551.950000,9449.070000,0.000000,0.166667,0.166667,0.000000,3.000000,7500.000000,13002.303930,410.800069,0.000000,10.000000,3.0
25%,2741.523040,1.000000,21958.090000,15636.290000,905.647500,1.000000,0.666667,0.541666,71.500000,12000.000000,22494.513405,567.772942,0.083333,12.000000,3.0
50%,3422.898250,1.000000,26008.730000,21224.335000,4828.500000,1.000000,1.000000,0.916667,107.500000,17000.000000,26604.720780,1171.389186,0.500000,12.000000,3.0
75%,6817.133432,1.000000,31609.457500,25470.657500,7874.782500,1.000000,1.000000,1.000000,161.500000,18000.000000,32272.496227,2790.351221,0.937500,12.000000,3.0
max,19043.138560,1.000000,49039.570000,40761.250000,15497.190000,1.000000,1.000000,1.000000,358.000000,30000.000000,50721.483360,18621.013310,1.000000,12.000000,3.0


In [58]:
c2 = data[data['Clusters']==2].describe()
c2

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,Clusters
count,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.0
mean,4972.492673,0.966673,4784.586766,3053.074444,1731.512321,0.813914,0.547870,0.635102,57.692460,11265.584416,6987.944228,2850.198253,0.186554,11.878968,2.0
std,3737.537209,0.116228,3574.562186,2887.523382,2336.698157,0.285540,0.357309,0.386208,55.051491,4337.436830,5298.417556,6101.832441,0.329341,0.693074,0.0
min,18.549421,0.090909,4.440000,0.000000,0.000000,0.083333,0.000000,0.000000,1.000000,1000.000000,92.865469,0.000000,0.000000,6.000000,2.0
25%,1828.521596,1.000000,1875.022500,716.752500,182.955000,0.737500,0.181818,0.267045,16.000000,8000.000000,3243.298355,571.357673,0.000000,12.000000,2.0
50%,4441.782967,1.000000,4375.310000,2371.050000,1031.950000,1.000000,0.583333,0.825757,45.000000,10975.000000,5802.340814,1380.444244,0.000000,12.000000,2.0
75%,7545.354754,1.000000,6903.190000,4474.397500,2236.850000,1.000000,0.916667,1.000000,82.000000,14125.000000,8887.309951,2792.350299,0.166667,12.000000,2.0
max,18495.558550,1.000000,22500.000000,14215.000000,22500.000000,1.000000,1.000000,1.000000,347.000000,30000.000000,40627.595240,76406.207520,1.000000,12.000000,2.0


In [59]:
c1 = data[data['Clusters']==1].describe()
c1

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,Clusters
count,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.000000,4362.0
mean,696.601552,0.848544,713.177815,355.726694,357.832536,0.601343,0.180726,0.460518,13.095369,2361.902924,954.051829,576.996682,0.181079,11.453003,1.0
std,804.614356,0.259743,809.940401,627.038349,493.146402,0.341919,0.256541,0.387545,14.599384,1212.850680,1081.207230,1603.619345,0.310043,1.428480,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,150.000000,0.000000,0.000000,0.000000,6.000000,1.0
25%,50.170105,0.818182,183.700000,0.000000,0.000000,0.250000,0.000000,0.000000,4.000000,1300.000000,288.064731,134.973089,0.000000,12.000000,1.0
50%,349.614601,1.000000,435.090000,69.410000,193.450000,0.625000,0.083333,0.416667,9.000000,2000.000000,578.049448,192.206510,0.000000,12.000000,1.0
75%,1133.017411,1.000000,947.907500,454.930000,485.197500,1.000000,0.250000,0.833333,16.000000,3000.000000,1229.193984,480.737899,0.222222,12.000000,1.0
max,4264.307261,1.000000,7323.380000,6454.250000,4249.920000,1.000000,1.000000,1.000000,186.000000,5700.000000,14229.882480,28483.254830,1.000000,12.000000,1.0


In [60]:
c0 = data[data['Clusters']==0].describe()
c0

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,Clusters
count,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.0
mean,1950.726268,0.911174,1389.360620,836.723413,553.100476,0.660740,0.360736,0.453395,21.023313,7742.453553,1860.369647,720.108089,0.192703,11.762401,0.0
std,2091.357947,0.198986,1293.977321,1001.432982,744.752038,0.337345,0.343426,0.393942,23.632924,2456.250947,1572.976565,1050.502450,0.326320,0.916675,0.0
min,0.000000,0.000000,0.010000,0.000000,0.000000,0.083333,0.000000,0.000000,1.000000,3500.000000,0.000000,0.000000,0.000000,6.000000,0.0
25%,169.853862,1.000000,393.090000,90.007500,0.000000,0.333333,0.083333,0.000000,6.000000,6000.000000,778.193486,173.661753,0.000000,12.000000,0.0
50%,1153.545848,1.000000,974.840000,463.420000,269.970000,0.750000,0.250000,0.416667,13.000000,7000.000000,1371.038092,315.388524,0.000000,12.000000,0.0
75%,3325.226944,1.000000,2043.057500,1203.822500,820.105000,1.000000,0.583333,0.833333,28.000000,9000.000000,2407.694824,961.241896,0.250000,12.000000,0.0
max,9321.555794,1.000000,7597.090000,6761.290000,6229.410000,1.000000,1.000000,1.000000,232.000000,20000.000000,9821.138115,24302.098190,1.000000,12.000000,0.0


### Group Comparison

In [119]:
#Displaying group totals and percentages in a table for reference. 
cluster_totals = pd.DataFrame({'Counts':list(cluster_count),
                              'Percentages': list(prc_)},
                             index = [1,0,2,3])
cluster_totals #Table is sorted by counts in descending order. 

,Counts,Percentages
1,4362,0.632
0,2016,0.292
2,504,0.073
3,24,0.003


In [122]:
#A preliminary scan of descriptive statistics above suggests 
#a productive comparison of the means across all dimensions of each group.
eval_ = pd.DataFrame({'Group C1 (4362)': round(c1.loc['mean'],2),
                      'Group C0 (2016)': round(c0.loc['mean'],2),
                     'Group C2 (504)': round(c2.loc['mean'],2),
                     'Group C3 (24)': round(c3.loc['mean'],2)},
                    index = data.columns[1:15])
eval_

,Group C1 (4362),Group C0 (2016),Group C2 (504),Group C3 (24)
BALANCE,696.60,1950.73,4972.49,5405.33
BALANCE_FREQUENCY,0.85,0.91,0.97,0.96
PURCHASES,713.18,1389.36,4784.59,27276.36
ONEOFF_PURCHASES,355.73,836.72,3053.07,21877.10
INSTALLMENTS_PURCHASES,357.83,553.10,1731.51,5399.26
PURCHASES_FREQUENCY,0.60,0.66,0.81,0.91
ONEOFF_PURCHASES_FREQUENCY,0.18,0.36,0.55,0.84
PURCHASES_INSTALLMENTS_FREQUENCY,0.46,0.45,0.64,0.72
PURCHASES_TRX,13.10,21.02,57.69,127.96
CREDIT_LIMIT,2361.90,7742.45,11265.58,16083.33


##### Group C03: Highest Value Customers

Group C03 is characterized by the highest means across many dimensions: balance, purchase values, purchase frequencies, purchase transactions, credit limit, payments, and percentage full payment. While comprising only .003 percent of the sample, they may comprise a majority or sizable minority of total value.

##### Group C02: High Value Customers

Group C02 is characterized by the 2nd highest means across many dimensions: balance, purchase values, purchase frequencies, purchase transactions, and credit limit. They comprise 7 percent of the sample but may comprise an outsized portion of value.

##### Group C00: Average Value Customers

Group C00 comprises 29 percent of the customer sample. The group is squarely in the middle between Groups 01 and 02 across nearly all dimensions. As they are a large minority, they may also comprise an outsized portion of total value. 

##### Group C01: Largest Group of Customers

Group C01 comprises 63% of the sample and is the largest pool of customers (4362). They are characterized by the lowest mean across many dimensions: balance, purchase values, purchase frequencies, purchase transactions, credit limit, payments, and percentage of full payments. Being the majority group, they are crucial for success.